#Classificação de textos com spaCy

##Etapa 1: Importação e instalação das bibliotecas

In [12]:
!pip install seaborn

In [2]:
import spacy
spacy.__version__

'3.7.4'

In [3]:
# Temporário por problemas no Colab
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
!python -m spacy download pt

⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import pandas as pd
import string
import spacy
import random
import seaborn as sns
import numpy as np

##Etapa 2: Carregamento da base de dados

In [6]:
base_dados = pd.read_csv('base_treinamento.txt', encoding= 'utf-8')

In [7]:
base_dados.shape


(196, 2)

In [8]:
base_dados.head()

,texto,emocao
0,este trabalho é agradável,alegria
1,gosto de ficar no seu aconchego,alegria
2,fiz a adesão ao curso hoje,alegria
3,eu sou admirada por muitos,alegria
4,adoro como você,alegria


In [9]:
base_dados.tail()

,texto,emocao
191,estou chocado e amedrontado com este assassina...,medo
192,é preciso afugenta com ímpeto este medo do inf...,medo
193,políticos usam suas forças para afugentar e am...,medo
194,o objetivo disso e apenas me amedrontar mais,medo
195,isso me apavora,medo


In [14]:
#sns.countplot(base_dados['emocao'], label = 'Contagem');

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

##Etapa 3: Funçãod para pré-processamento dos textos

In [15]:
pontuacoes = string.punctuation
pontuacoes

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
from spacy.lang.pt.stop_words import STOP_WORDS
stop_words = STOP_WORDS

In [17]:
print(stop_words)

{'muito', 'estão', 'final', 'tais', 'nenhuma', 'certamente', 'máximo', 'sou', 'dentro', 'segundo', 'porquê', 'coisa', 'embora', 'corrente', 'na', 'seis', 'maioria', 'longe', 'naquele', 'devem', 'tarde', 'meus', 'nova', 'baixo', 'qual', 'nada', 'sob', 'tua', 'vindo', 'contra', 'do', 'segunda', 'contudo', 'você', 'esse', 'novo', 'se', 'deverá', 'fará', 'três', 'aquilo', 'estes', 'nessa', 'estivestes', 'oitavo', 'daquela', 'tanto', 'tentaram', 'no', 'ponto', 'dessa', 'ambas', 'essas', 'sexto', 'comprida', 'todos', 'apenas', 'umas', 'teve', 'oitava', 'suas', 'estar', 'lugar', 'des', 'estas', 'dá', 'veja', 'tu', 'pelos', 'vez', 'já', 'eu', 'desde', 'seria', 'cima', 'cada', 'da', 'estás', 'qualquer', 'dezoito', 'podia', 'aí', 'nunca', 'nos', 'terceira', 'sétima', 'ela', 'nós', 'portanto', 'favor', 'desta', 'número', 'vossas', 'vinda', 'estará', 'aos', 'geral', 'tiveste', 'vossos', 'poderá', 'doze', 'somente', 'lado', 'pegar', 'quatro', 'vosso', 'tentei', 'nuns', 'cento', 'tentar', 'tenho', '

In [18]:
len(stop_words)

416

In [19]:
pln = spacy.load('pt_core_news_sm')

In [20]:
pln

In [21]:
def preprocessamento(texto):
  texto = texto.lower()
  documento = pln(texto)
  lista = []
  for token in documento:
    #lista.append(token.text)
    lista.append(token.lemma_)

  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in pontuacoes]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

  return lista


In [22]:
teste = preprocessamento('Estou aPrendendo 1 10 23 processamento de linguagem natural, curso em Curitiba')
teste

'aprender processamento linguagem natural curso curitiba'

##Etapa 4: Pré-processamento da base de dados

###Limpeza dos textos

In [23]:
base_dados.head(10)

,texto,emocao
0,este trabalho é agradável,alegria
1,gosto de ficar no seu aconchego,alegria
2,fiz a adesão ao curso hoje,alegria
3,eu sou admirada por muitos,alegria
4,adoro como você,alegria
5,adoro seu cabelo macio,alegria
6,adoro a cor dos seus olhos,alegria
7,somo tão amáveis um com o outro,alegria
8,sinto uma grande afeição por ele,alegria
9,quero agradar meus filhos,alegria


In [24]:
base_dados['texto'] = base_dados['texto'].apply(preprocessamento)

In [25]:
base_dados.head(10)

,texto,emocao
0,trabalho agradável,alegria
1,gosto ficar em o aconchego,alegria
2,fiz adesão a o curso hoje,alegria
3,admirar,alegria
4,adorar,alegria
5,adorar cabelo macio,alegria
6,adorar cor de o olho,alegria
7,somo amável outro,alegria
8,sinto afeição,alegria
9,querer agradar filho,alegria


###Tratamento de classe

In [26]:
exemplo_base_dados = [["este trabalho é agradável", {"ALEGRIA": True, "MEDO": False}],
                      ["este lugar continua assustador", {"ALEGRIA": False, "MEDO": True}]]

In [27]:
type(exemplo_base_dados)

list

In [28]:
exemplo_base_dados[0]

['este trabalho é agradável', {'ALEGRIA': True, 'MEDO': False}]

In [29]:
exemplo_base_dados[0][0]

'este trabalho é agradável'

In [ ]:
exemplo_base_dados[0][1]

In [31]:
type(exemplo_base_dados[0][1])


dict

In [32]:
base_dados_final = []
i = 0
for texto, emocao in zip(base_dados['texto'], base_dados['emocao']):
  if emocao == 'alegria':
    dic = ({'ALEGRIA': True, 'MEDO': False})
  elif emocao == 'medo':
    dic = ({'ALEGRIA': False, 'MEDO': True})

  base_dados_final.append([texto, dic.copy()])

In [33]:
len(base_dados_final)

196

In [34]:
base_dados_final[0]

['trabalho agradável', {'ALEGRIA': True, 'MEDO': False}]

In [35]:
base_dados_final[0][0]

'trabalho agradável'

In [36]:
base_dados_final[0][1]

{'ALEGRIA': True, 'MEDO': False}

In [37]:
type(base_dados_final[0][1])

dict

In [38]:
base_dados_final

[['trabalho agradável', {'ALEGRIA': True, 'MEDO': False}],
 ['gosto ficar em o aconchego', {'ALEGRIA': True, 'MEDO': False}],
 ['fiz adesão a o curso hoje', {'ALEGRIA': True, 'MEDO': False}],
 ['admirar', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar cabelo macio', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar cor de o olho', {'ALEGRIA': True, 'MEDO': False}],
 ['somo amável outro', {'ALEGRIA': True, 'MEDO': False}],
 ['sinto afeição', {'ALEGRIA': True, 'MEDO': False}],
 ['querer agradar filho', {'ALEGRIA': True, 'MEDO': False}],
 ['sinto completamente amar', {'ALEGRIA': True, 'MEDO': False}],
 ['amo', {'ALEGRIA': True, 'MEDO': False}],
 ['alívio', {'ALEGRIA': True, 'MEDO': False}],
 ['dor amenizar finalmente', {'ALEGRIA': True, 'MEDO': False}],
 ['achar apaixoneir', {'ALEGRIA': True, 'MEDO': False}],
 ['amar maravilhoso', {'ALEGRIA': True, 'MEDO': False}],
 ['sentir animar', {'ALEGRIA': True, 'MEDO': False}],
 ['sintar hoje', {'ALEGRIA': Tr

##Etapa 5: Criação do classificador

In [39]:
from spacy.training import Example

In [40]:
modelo = spacy.blank('pt')
textcat = modelo.add_pipe("textcat")
textcat.add_label("ALEGRIA")
textcat.add_label("MEDO")
historico = []